In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
input_file_name = 'data-v2.xlsx'
file_path = os.path.join(os.getcwd(), 'input-files', input_file_name)
read_columns = ['Time_stamp', 'SOLAR_GHI_GROUND_01_AVG', 'SOLAR_DIRECTRADIATION_GROUND_01_AVG', 'SOLAR_DIFFUSERADIATION_GROUND_01_AVG', 'DAY_OF_YEAR', 'DECLINATION', 'EOT', 'TC', 'LST', 'HOUR_ANGLE', 'ZENITH_ANGLE']
data_DF = pd.read_excel(file_path, usecols=read_columns)
data_DF.head()

,Time_stamp,SOLAR_DIFFUSERADIATION_GROUND_01_AVG,SOLAR_DIRECTRADIATION_GROUND_01_AVG,SOLAR_GHI_GROUND_01_AVG,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097


In [3]:
# data_DF.head()
# data_DF.dtypes
# data_DF.isnull().sum()
if data_DF.isnull().values.any():
    data_DF = data_DF.dropna()
# data_DF.duplicated().sum()
# data_DF.describe()

In [4]:
# Calculation for I0n
# Extraterrestrial irradiance on a normal surface 
# I0n = Isc(1 + 0.033 * cos( 360 * n/ 365))
# Isc​ is the solar constant (1367 W/m²)
# n is the day of the year (from the date in your period column)

data_DF['I0n'] = 1367 * (1 + 0.033 * np.cos(2 * np.pi * data_DF['DAY_OF_YEAR'] / 365))
data_DF['COS_ZENITH'] = np.cos(np.deg2rad(data_DF['ZENITH_ANGLE']))
data_DF.rename(columns={
    'SOLAR_GHI_GROUND_01_AVG': 'GHI', 
    'SOLAR_DIRECTRADIATION_GROUND_01_AVG': 'DNI', 
    'SOLAR_DIFFUSERADIATION_GROUND_01_AVG': 'DHI'
}, inplace=True)
data_DF['Kt'] = data_DF['GHI'] / (data_DF['I0n'] * data_DF['COS_ZENITH'])
data_DF['Kd'] = data_DF['DHI'] / data_DF['GHI']
data_DF['Kn'] = data_DF['DNI'] / data_DF['I0n']

# # Horizontal extraterrestrial irradiance G0h
data_DF['G0h'] = data_DF['I0n'] * data_DF['COS_ZENITH']
# data_DF.head()
data_DF.describe()

c:\Users\zeesh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h
count,43843,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,2.276500e+04,43843.000000,43843.000000
mean,2012-03-20 18:13:28.521314816,96.483501,138.949679,191.801936,167.135483,-0.055212,-1.219133,-0.385252,11.540863,-6.887056,90.036603,1368.608011,-0.000458,0.256163,inf,0.101521,-5.558835
min,2011-10-20 12:20:00,0.000000,0.000000,0.000000,1.000000,-23.449783,-14.256968,-0.602549,-0.602549,-189.038242,5.060667,1321.890671,-0.996179,-56.609481,0.000000e+00,0.000000,-1406.313308
25%,2012-01-04 15:35:00,0.000000,0.000000,0.000000,77.000000,-18.547676,-6.583959,-0.474666,5.538352,-96.924718,55.302632,1331.438771,-0.570104,-0.000000,4.239986e-01,0.000000,-772.619052
50%,2012-03-20 18:20:00,1.306513,0.222270,1.407574,153.000000,-0.403653,-2.453458,-0.405824,11.550203,-6.746955,89.960210,1375.681683,0.000694,0.000000,6.111101e-01,0.000161,0.923080
75%,2012-06-04 21:05:00,188.241005,265.771805,364.054611,229.000000,18.547676,3.559295,-0.305612,17.540239,83.103579,124.757450,1402.798809,0.569242,0.546651,9.043194e-01,0.194240,783.296622
max,2012-08-19 23:50:00,645.252136,869.604736,1069.615234,365.000000,23.449783,16.379094,-0.091948,23.741385,176.120773,174.989403,1412.111000,0.996102,167.828181,inf,0.629050,1319.657802
std,NaN,127.774209,213.271242,271.433163,108.308637,17.894359,8.694306,0.144905,6.930225,103.953373,41.715312,34.214096,0.609564,1.665176,NaN,0.155693,834.526351


In [5]:
# K-tests
k_test_condition_1 = (data_DF['GHI'] > 50) & (data_DF['ZENITH_ANGLE'] < 75) & (data_DF['Kd'] < 1.05)
k_test_condition_2 = (data_DF['GHI'] > 50) & (data_DF['ZENITH_ANGLE'] > 75) & (data_DF['Kd'] < 1.10)
k_test_condition_3 = (data_DF['Kn'] < 0.8) & (data_DF['Kn'] > 0)
k_test_condition_4 = (data_DF['Kd'] < 0.96) & (data_DF['Kt'] > 0.6)

data_DF['k_test'] = np.where(
    k_test_condition_1 | k_test_condition_2 | k_test_condition_3 | k_test_condition_4,
    'Passed',
    'Failed'
)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685,1381.676507,0.766093,0.616429,0.750141,0.301014,1058.492488,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586,1381.676507,0.763068,0.549012,0.976274,0.309041,1054.313067,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670,1381.676507,0.758411,0.470246,0.973463,0.237218,1047.878899,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658,1381.676507,0.752131,0.457770,0.517614,0.221697,1039.202231,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097,1381.676507,0.744240,0.469357,0.854338,0.065544,1028.299580,Passed


In [6]:
#  Individual Limits Test (ILT)
ILT_condition_1 = (data_DF['GHI'] > -4) & (data_DF['GHI'] < (1.5 * data_DF['I0n'] * data_DF['COS_ZENITH']**1.2 + 100))
ILT_condition_2 = (data_DF['DHI'] > -4) & (data_DF['DHI'] < (0.97 * data_DF['I0n'] * data_DF['COS_ZENITH']**1.2 + 50))
ILT_condition_3 = (data_DF['DNI'] > -4) & (data_DF['DNI'] < data_DF['I0n'])
ILT_condition_4 = (data_DF['DHI'] < 0.8 * data_DF['G0h'])
ILT_condition_5 = (data_DF['GHI'] - data_DF['DHI'] < data_DF['G0h'])

data_DF['individual_limit_test'] = np.where(
    ILT_condition_1 | ILT_condition_2 | ILT_condition_3 | ILT_condition_4 | ILT_condition_5,
    'Passed',
    'Failed'
)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test,individual_limit_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685,1381.676507,0.766093,0.616429,0.750141,0.301014,1058.492488,Passed,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586,1381.676507,0.763068,0.549012,0.976274,0.309041,1054.313067,Passed,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670,1381.676507,0.758411,0.470246,0.973463,0.237218,1047.878899,Passed,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658,1381.676507,0.752131,0.457770,0.517614,0.221697,1039.202231,Passed,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097,1381.676507,0.744240,0.469357,0.854338,0.065544,1028.299580,Passed,Passed


In [7]:
# Night-Time Zero Testing
filter_condition_1 = data_DF['GHI'] > 5
filter_condition_2 = data_DF['ZENITH_ANGLE'] < 85
data_DF['night_time_test'] = np.where(
    filter_condition_1 | filter_condition_2,
    'Passed',
    'Failed'
)
output_file_name = 'output-v2.csv'
output_path = os.path.join(os.getcwd(), 'output-files', output_file_name)
data_DF.to_csv(output_path, encoding='utf-8', index=None)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test,individual_limit_test,night_time_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685,1381.676507,0.766093,0.616429,0.750141,0.301014,1058.492488,Passed,Passed,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586,1381.676507,0.763068,0.549012,0.976274,0.309041,1054.313067,Passed,Passed,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670,1381.676507,0.758411,0.470246,0.973463,0.237218,1047.878899,Passed,Passed,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658,1381.676507,0.752131,0.457770,0.517614,0.221697,1039.202231,Passed,Passed,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097,1381.676507,0.744240,0.469357,0.854338,0.065544,1028.299580,Passed,Passed,Passed
